In [1]:
%load_ext cython

In [2]:
import numpy as np
from scipy.misc import electrocardiogram

In [3]:
x1d = electrocardiogram()
x3d = x1d.reshape((1, -1, 1))

In [4]:
def sampen(L, m, r):
    N = len(L)
    B = 0.0
    A = 0.0
    
    
    # Split time series and save all templates of length m
    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    # Similar for computing A
    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    return -np.log(A / B)

In [5]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double, intc, array
from libc.math cimport log


def SampleEntropy(const double[:, :, :] signal, int M, double r):
    cdef Py_ssize_t n = signal.shape[1], k = signal.shape[2], p = signal.shape[0]
    entropy = zeros((p, M, k), dtype=npy_double)
    cdef double[:, :, ::1] ent = entropy
    cdef long nj, j
    cdef long[:] run = zeros(n, dtype=int), lastrun = zeros(n, dtype=int)
    cdef long N, M1
    cdef double[:, :, :] A = zeros((p, M, k), dtype=npy_double), B = zeros((p, M, k), dtype=npy_double)
    cdef double Y1

    cdef Py_ssize_t i, jj, mm, kk, axis, wind
    for wind in range(p):
        for axis in range(k):
            run[:] = 0
            lastrun[:] = 0
            for i in range(n-1):
                nj = n - i - 1
                Y1 = signal[wind, i, axis]
                for jj in range(nj):
                    j = jj + i + 1
                    if (((signal[wind, j, axis] - Y1) < r) and ((Y1 - signal[wind, j, axis]) < r)):
                        run[jj] = lastrun[jj] + 1
                        M1 = M if (M < run[jj]) else run[jj]
                        for mm in range(M1):
                            A[wind, mm, axis] += 1
                            if (j < (n - 1)):
                                B[wind, mm, axis] += 1
                    else:
                        run[jj] = 0
                for kk in range(nj):
                    lastrun[kk] = run[kk]

    N = <long>(n * (n - 1) / 2)

    for wind in range(p):
        for axis in range(k):
            ent[wind, 0, axis] = -log(A[wind, 0, axis] / N)

            for mm in range(1, M):
                ent[wind, mm, axis] = -log(A[wind, mm, axis] / B[wind, mm - 1, axis])

    return entropy

def sent_1d(const double[:] signal, int M, double r):
    cdef Py_ssize_t n = signal.size
    entropy = zeros(M, dtype=npy_double)
    cdef double[:] ent = entropy
    cdef long nj, j
    cdef long[:] run = zeros(n, dtype=int), lastrun = zeros(n, dtype=int)
    cdef long N, M1
    cdef double[:] A = zeros(M, dtype=npy_double), B = zeros(M, dtype=npy_double)
    cdef double Y1

    cdef Py_ssize_t i, jj, mm, kk, axis, wind

    run[:] = 0
    lastrun[:] = 0
    for i in range(n-1):
        nj = n - i - 1
        Y1 = signal[i]
        for jj in range(nj):
            j = jj + i + 1
            if (((signal[j] - Y1) < r) and ((Y1 - signal[j]) < r)):
                run[jj] = lastrun[jj] + 1
                M1 = M if (M < run[jj]) else run[jj]
                for mm in range(M1):
                    A[mm] += 1
                    if (j < (n - 1)):
                        B[mm] += 1
            else:
                run[jj] = 0
        for kk in range(nj):
            lastrun[kk] = run[kk]

    N = <long>(n * (n - 1) / 2)

    ent[0] = -log(A[0] / N)

    for mm in range(1, M):
        ent[mm] = -log(A[mm] / B[mm - 1])

    return entropy

In [6]:
from sampent import sampleentropy as fsampleentropy

In [15]:
np.random.seed(5)
x = np.around(np.random.rand(50000, 150, 3), 2)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [16]:
# np.allclose(SampleEntropy(x, 3, 0.5)[:, -1, :], fsampleentropy(xf, 3, 0.5).T)

True

In [17]:
# %timeit SampleEntropy(x, 3, 0.5)
print('~16.9s (38.9ms) per loop')

16.9 s ± 38.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit fsampleentropy(xf, 3, 0.5)

8.67 s ± 174 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
L = 1
r = 0.3

print(sampen(x[0, :, 0], L, r))
print(sent_1d(x[0, :, 0], L, r)[-1])
print(fsampen(x[0, :, 0], L, r)[-1])
print(fsampen2(x[0, :, 0], L, r))
# print(SampleEntropy(x, 5, 0.3)[0, :, 0])
# print(fsampleentropy(xf, 5, 0.3))

0.74806293815606
0.740302318477223
0.740302318477223
0.740302318477223


In [13]:
%timeit sent_1d(x[0, :, 0], 4, 0.3)
%timeit fsampen(x[0, :, 0], 4, 0.3)
%timeit fsampen2(x[0, :, 0], 4, 0.3)

41.1 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.7 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.58 µs ± 74.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [2]:
import numpy as np

In [4]:
np.random.seed(1)
print(np.around(np.random.rand(10), 2))

[0.42 0.72 0.   0.3  0.15 0.09 0.19 0.35 0.4  0.54]


In [102]:
y = x[0, :, 0]
n = y.size
run = np.zeros(n, dtype='int')
lastrun = np.zeros(n, dtype='int')
print(y, '\n')

print('  i   jj  j   y1     y[j]     d<0.3  run[jj]')
for i in range(n-1):
    y1 = y[i]
    for jj in range(n-i-1):
        j = jj + i + 1
        if np.abs(y[j] - y1) < 0.3:
            run[jj] = lastrun[jj] + 1
        else:
            run[jj] = 0
        print(f'{i:3d}{jj:4d}{j:4d}{y1:7.2f}{y[j]:7.2f}{np.abs(y[j] - y1) < 0.3:6b}{run[jj]:7d}')
    for jj in range(n-i-1):
        lastrun[jj] = run[jj]

[0.22 0.87 0.21 0.92 0.49 0.61 0.77 0.52 0.3  0.19] 

  i   jj  j   y1     y[j]     d<0.3  run[jj]
  0   0   1   0.22   0.87     0      0
  0   1   2   0.22   0.21     1      1
  0   2   3   0.22   0.92     0      0
  0   3   4   0.22   0.49     1      1
  0   4   5   0.22   0.61     0      0
  0   5   6   0.22   0.77     0      0
  0   6   7   0.22   0.52     0      0
  0   7   8   0.22   0.30     1      1
  0   8   9   0.22   0.19     1      1
  1   0   2   0.87   0.21     0      0
  1   1   3   0.87   0.92     1      2
  1   2   4   0.87   0.49     0      0
  1   3   5   0.87   0.61     1      2
  1   4   6   0.87   0.77     1      1
  1   5   7   0.87   0.52     0      0
  1   6   8   0.87   0.30     0      0
  1   7   9   0.87   0.19     0      0
  2   0   3   0.21   0.92     0      0
  2   1   4   0.21   0.49     1      3
  2   2   5   0.21   0.61     0      0
  2   3   6   0.21   0.77     0      0
  2   4   7   0.21   0.52     0      0
  2   5   8   0.21   0.30     1      1
  2 

In [73]:
sent_1d(y, 3, 0.3)

[20.  9.  4.] [17.  8.  3.]
[5 0 0 0 0 0 1 0 1 0] [5 0 0 0 0 0 1 0 1 0]


array([0.81093022, 0.63598877, 0.69314718])

In [74]:
from scipy.spatial.distance import chebyshev
chebyshev(y[2:6], y[1:5])

0.7100000000000001

In [76]:
res = np.zeros((9, 9))
for i in range(9):
    for j in range(9):
        res[i, j] = chebyshev(y[i:i+2], y[j:j+2])

(res < 0.3).sum() - 9

18

In [75]:
res = np.zeros((8, 8))
for i in range(8):
    for j in range(8):
        res[i, j] = chebyshev(y[i:i+3], y[j:j+3])

(res < 0.3).sum() - 8

8

In [71]:
np.log(9/17)

-0.6359887667199967